In [11]:
import numpy as np
table26 = np.array([
             [[0.032,0.024],[0.032,0.024],[0.032,0.024],[0.032,0.024],[0.032,0.024],[0.032,0.024],[0.032,0.024],[0.032,0.024],[0.032,0.024]],
             [[0.037,0.028],[0.043,0.032],[0.048,0.036],[0.051,0.039],[0.055,0.041],[0.057,0.044],[0.064,0.048],[0.068,0.052],[0.037,0.028]],
             [[0.037,0.028],[0.044,0.033],[0.052,0.039],[0.057,0.044],[0.063,0.047],[0.067,0.051],[0.077,0.059],[0.085,0.065],[0.037,0.028]],
             [[0.047,0.035],[0.053,0.040],[0.060,0.045],[0.065,0.049],[0.071,0.053],[0.075,0.056],[0.084,0.063],[0.091,0.069],[0.047,0.035]],
             [[0.045,0.035],[0.049,0.037],[0.052,0.040],[0.056,0.043],[0.059,0.044],[0.060,0.045],[0.065,0.049],[0.069,0.052],[0.000,0.035]],
             [[0.000,0.035],[0.000,0.043],[0.000,0.051],[0.000,0.057],[0.000,0.063],[0.000,0.068],[0.000,0.080],[0.000,0.088],[0.045,0.035]],
             [[0.057,0.043],[0.064,0.048],[0.071,0.053],[0.076,0.057],[0.080,0.060],[0.084,0.064],[0.091,0.069],[0.097,0.073],[0.000,0.043]],
             [[0.000,0.043],[0.000,0.051],[0.000,0.059],[0.000,0.065],[0.000,0.071],[0.000,0.076],[0.000,0.087],[0.000,0.096],[0.057,0.043]],
             [[0.000,0.056],[0.000,0.064],[0.000,0.072],[0.000,0.079],[0.000,0.085],[0.000,0.089],[0.000,0.100],[0.000,0.107],[0.000,0.056]]
             ])

ratio_length_depth = np.array([32,32,32,30,30,30,28,28,28])
length_room = float(input("enter the length of the room(in m): "))*1000
breadth_room = float(input("enter the breadth of the room(in m): "))*1000

# determining whether length or breadth is bigger and making breadth bigger
if(length_room>breadth_room):
    temp = length_room
    length_room = breadth_room 
    breadth_room = temp


width_column = float(input("enter the width of column(in mm): "))
diameter_rod = float(input("diameter of rod you want to use in mm: "))
floor_finish = float(input("enter the floor finish pressure(in KN/m*m): "))
live_load = float(input("enter the live load to be considered(in KN/m*m): "))
fck = float(input("enter the fck value(in N/mm*mm): "))
case = int(input("enter the case number in table(1-9): "))
case = case-1


# finding out depth of concrete required by first calculating depth and adding diameter of rod and 15mm of 
# mortar above it

depth_concrete_initial = (length_room / ratio_length_depth[case])
print(f"\n\nInitial depth we get is {depth_concrete_initial} mm")
depth_concrete = depth_concrete_initial + 15 + (diameter_rod/2)
print(f"Depth we get after taking rod and mortar in consideration is {depth_concrete} mm")

# checking whether give column width will support the slab or not

if((length_room + depth_concrete_initial) > (length_room + width_column) or (breadth_room + depth_concrete_initial - diameter_rod)>(breadth_room + width_column)):
    print("\n\n\nWarning!!!!!!,You need to increase column width and Values below doesn't matter any more\n\n\n") 

# defining fuction to generate values with the help of table26

def information(a):
    index = int(np.floor((a-1)*10))
    alpha_x_negative = table26[case][index][0] + ((table26[case][index+1][0] - table26[case][index][0])/0.1)*(a - (1 + (index/10)))
    alpha_x_positive = table26[case][index][1] + ((table26[case][index+1][1] - table26[case][index][1])/0.1)*(a - (1 + (index/10)))
    alpha_y_negative = table26[case][8][0]
    alpha_y_positive = table26[case][8][1]
    alpha_max = max(alpha_x_positive,alpha_x_negative,alpha_y_positive,alpha_y_negative)
    print(f"alpha_x_negative:{round(alpha_x_negative,3)}")
    print(f"alpha_x_positive:{round(alpha_x_positive,3)}")
    print(f"alpha_y_negative:{round(alpha_y_negative,3)}")
    print(f"alpha_y_positive:{round(alpha_y_positive,3)}")
    return (alpha_max)


#calcultaing total load and factorial load which would be helpful further

total_load = floor_finish * 1 + live_load * 1 + 25*1*(depth_concrete/1000) # considering 1 meter of slab and 25 is concrete
# gamma coefficient
factorial_load = 1.5 * total_load #1.5 is safety factor
print(f"total load is {round(total_load,3)}KN/m and factorial load is {round(factorial_load,3)}KN/m")

#calcultaing ratio to generate values for table using defined function

a = (breadth_room + depth_concrete_initial - diameter_rod)/(length_room + depth_concrete_initial)

if(a<2):
    information(a)
    # calculating moment and checking value of depth we got

    moment_max = information(a) * factorial_load * (length_room**2)

    # for 415fe steel x_u = 0.48*temporary
    depth_temp = np.sqrt(moment_max/(0.138*fck*1000))
    if(depth_temp > depth_concrete):
        depth_concrete = depth_temp
        print(f"\n\nnew depth is {depth_concrete}\n\n")
else:
    print("Oops!!!, Sorry, we can't calculate for the dimensons you have given")
    


enter the length of the room(in m): 5
enter the breadth of the room(in m): 4
enter the width of column(in mm): 230
diameter of rod you want to use in mm: 10
enter the floor finish pressure(in KN/m*m): 1.2
enter the live load to be considered(in KN/m*m): 4
enter the fck value(in N/mm*mm): 20
enter the case number in table(1-9): 2
Initial depth we get is 125.0 mm
Depth we get after taking rod and mortar in consideration is 145.0 mm
total load is 8.825KN/m and factorial load is 13.237KN/m
alpha_x_negative:0.049
alpha_x_positive:0.037
alpha_y_negative:0.037
alpha_y_positive:0.028
alpha_x_negative:0.049
alpha_x_positive:0.037
alpha_y_negative:0.037
alpha_y_positive:0.028
61.4456281389596
